# Amazon electronics dataset exploration

## 2018 Amazon Review Data

A subset of the Amazon Review Data (2018), the electronics category data is roughly 20M engagements from Amazon users.  

*Source*: Justifying recommendations using distantly-labeled reviews and fined-grained aspects
Jianmo Ni, Jiacheng Li, Julian McAuley
Empirical Methods in Natural Language Processing (EMNLP), 2019, https://nijianmo.github.io/amazon/index.html

In [ ]:
!ls ../data/2018

In [ ]:
import pandas as pd
df = pd.read_csv('../data/2018/Electronics.csv', nrows=100000, names=["item", "user", "rating", "timestamp"])

Ratings only: These datasets include no metadata or reviews, but only (item,user,rating,timestamp) tuples. Thus they are suitable for use with mymedialite (or similar) packages.

In [ ]:
df.head()

In [ ]:
len(df)

Hmm... are the four columns sufficient for our system? Can we infer a purchase based on the presence of a rating? Do we assume a user with no rating for a product failed to purchase? Yeesh... that doesn't seem supportable. I guess the prediction here is not whether they bought it but whether they were motivated to source a review. Here the review becomes the reward, not the sale ... go off and read the paper: https://cseweb.ucsd.edu/~jmcauley/pdfs/emnlp19a.pdf

In [ ]:
df.describe()

In [ ]:
df.item.value_counts()

## 2023 Amazon Reviews Data

### Preprocessing

In [ ]:
!ls -lh ../data/2023

2023 publication, see https://amazon-reviews-2023.github.io/

In [1]:
import json 
import pandas as pd

In [2]:
reviews = pd.read_json('../data/2023/Electronics.jsonl', lines=True, nrows=100, )

In [3]:
reviews.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,3,Smells like gasoline! Going back!,First & most offensive: they reek of gasoline ...,[{'small_image_url': 'https://m.media-amazon.c...,B083NRGZMM,B083NRGZMM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-07-18 22:58:37.948,0,True
1,1,Didn’t work at all lenses loose/broken.,These didn’t work. Idk if they were damaged in...,[],B07N69T6TM,B07N69T6TM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-06-20 18:42:29.731,0,True
2,5,Excellent!,I love these. They even come with a carry case...,[],B01G8JO5F2,B01G8JO5F2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2018-04-07 09:23:37.534,0,True
3,5,Great laptop backpack!,I was searching for a sturdy backpack for scho...,[],B001OC5JKY,B001OC5JKY,AGGZ357AO26RQZVRLGU4D4N52DZQ,2010-11-20 18:41:35.000,18,True
4,5,Best Headphones in the Fifties price range!,I've bought these headphones three times becau...,[],B013J7WUGC,B07CJYMRWM,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,2023-02-17 02:39:41.238,0,True


In [ ]:
# We need to reduce the size of this dataset or risk blowing our memory budget, filter down to essentials for our prediction task 
!cd ../data/2023 && jq -c '{rating, parent_asin, user_id, timestamp}' Electronics.jsonl > reviews_43M.jsonl

In [ ]:
reviews = pd.read_json('../data/2023/Electronics_min.jsonl', lines=True)

In [ ]:
reviews.to_parquet("../data/2023/Electronics_min.parquet")

In [ ]:
reviews.iloc[0]

In [ ]:
len(reviews.user_id.unique())

In [ ]:
reviews.hist()

In [ ]:
items = pd.read_json("../data/2023/meta_Electronics.jsonl", lines=True, nrows=100) 

In [ ]:
items.head()

In [ ]:
# Filter down to essential fields
!cd ../data/2023 && jq -c '{title, average_rating, description, price, images, rating_number, parent_asin}' meta_Electronics.jsonl > meta_Electronics_min.jsonl

In [ ]:
import pandas as pd

In [ ]:
# Note this for whatever reason burns about 30G of RAM during the load, even though the json is only 2.8G uncompressed, we should get this into a parquet file stat
items = pd.read_json("../data/2023/meta_Electronics.jsonl", lines=True)

In [ ]:
items.drop(['main_category', 'features', 'videos', 'store', 'categories', 'details', 'bought_together', 'subtitle', 'author'], axis=1, inplace=True)

In [ ]:
items.head()

In [ ]:
items.price = items.price.astype(str)

In [ ]:
items.to_parquet("../data/2023/meta_Electronics.parquet")

In [ ]:
len(items)

In [ ]:
items.hist()

In [ ]:
items.iloc[0]

In [ ]:
# Per the dataset documentation: Note: Products with different colors, styles, sizes usually belong to the same parent ID. 
# The “asin” in previous Amazon datasets is actually parent ID. Please use parent ID to find product meta.
item = reviews.iloc[5].parent_asin
items[items.parent_asin == item]

In [ ]:
reviews[reviews.parent_asin == items.iloc[1].parent_asin]

I can't load the entirety of the reviews in one shot... but I can fit every item in memory. So every review will be grounded to an item, but many reviews will be hidden. I don't think this matters for this project. If I want to fit more reviews, I can simply preprocess the data to rejct unneeded fields and (notably text fields) and dramatically reduce memory requirements. I can alternatively load only the critical columns, yes? 

In [6]:
reviews = pd.read_parquet("../data/2023/reviews_10M.parquet")
items = pd.read_parquet("../data/2023/items_1.6M.parquet")

In [7]:
reviews

,rating,parent_asin,user_id,timestamp
0,3,B083NRGZMM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2022-07-18 22:58:37.948
1,1,B07N69T6TM,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2020-06-20 18:42:29.731
2,5,B01G8JO5F2,AFKZENTNBQ7A7V7UXW5JJI6UGRYQ,2018-04-07 09:23:37.534
3,5,B001OC5JKY,AGGZ357AO26RQZVRLGU4D4N52DZQ,2010-11-20 18:41:35.000
4,5,B07CJYMRWM,AG2L7H23R5LLKDKLBEF2Q3L2MVDA,2023-02-17 02:39:41.238
...,...,...,...,...
9999995,4,B0BR6FBPD9,AHG2SVUXNKVWEHVTVRZHIDRCWFQA,2021-07-07 17:10:50.418
9999996,5,B07DVRGM9M,AHG2SVUXNKVWEHVTVRZHIDRCWFQA,2018-07-19 14:04:49.157
9999997,5,B0B3TJMN8P,AGT6T3SMU577AW4KZOWCX7VIY3IQ,2022-08-26 16:29:27.240
9999998,5,B0B3158RQ2,AFY2KJ5YAVB77AAACLXEAHT4CHWQ,2022-12-06 20:21:24.745


In [10]:
users = reviews.groupby(['user_id']).rating.count()
users = pd.DataFrame(users).reset_index()
users.rename(columns={'rating':'ratings'}, inplace=True)

In [13]:
users


,user_id,ratings
0,AE2222FRPDMNOMYOMCWIANTXP7UQ,3
1,AE22236AFRRSMQIKGG7TPTB75QEA,21
2,AE2225IRFZTQ77QWCB4UVSFQGS5A,1
3,AE2227JUF7JR4U5YO22WRJXI7YPA,1
4,AE222CLD7MMLOFP37THNLRRPBGZA,2
...,...,...
2294445,AHZZZWVHLYTQU55PD4FJULUEKTXA,17
2294446,AHZZZY2XVWEUJUTYPGGL4WXH6CSA,20
2294447,AHZZZY4DFLAWPBQYFQFWVACNGURA,3
2294448,AHZZZZPE45DYV2WZ2MYXZRHWSEKA,9


In [14]:
users_small = users[users.ratings > 10]
users_small

,user_id,ratings
1,AE22236AFRRSMQIKGG7TPTB75QEA,21
7,AE222H3FGXWLHRFUMGMS2RR57NDQ,16
27,AE223HSAZU6TUGSD7F47WOADLS2Q,14
45,AE224LWIR4ZDJLRKORFHMJGABP6Q,21
53,AE2254CLKQFGLVCWHFRAMSW4CNZQ,22
...,...,...
2294403,AHZZXP52C2AFASKIZR44MMSPNNNA,12
2294416,AHZZY6MRMU6PA6YHUWPHNRFO47QQ,11
2294432,AHZZYZ2ZNIDOJKRXALZBIX7VVKYA,11
2294445,AHZZZWVHLYTQU55PD4FJULUEKTXA,17


In [16]:
# narrow down reviews, too many to operate on with limited resources
reviews_small = reviews[reviews.user_id.isin(users_small.user_id.unique())]
reviews_small

,rating,parent_asin,user_id,timestamp
45,5,B08912RRG5,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,2022-05-22 20:42:27.409
46,5,B07GVWH35S,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,2019-06-08 21:07:32.317
47,5,B06VTQK7CB,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,2019-06-08 17:16:26.774
48,1,B0776DSP7Y,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,2018-05-12 00:32:47.886
49,2,B00NWXM5U0,AHITBJSS7KYUBVZPX7M2WJCOIVKQ,2015-05-23 01:48:59.000
...,...,...,...,...
9999878,5,B00H8HMOD4,AFHZUWRCTRXAZNE2DOO36IST5BVA,2014-07-09 14:22:56.000
9999879,3,B00CRTRBK4,AFHZUWRCTRXAZNE2DOO36IST5BVA,2014-07-09 14:20:48.000
9999880,1,B00ATSKZH6,AFHZUWRCTRXAZNE2DOO36IST5BVA,2013-12-03 23:23:44.000
9999881,5,B0043D2L70,AFHZUWRCTRXAZNE2DOO36IST5BVA,2013-08-17 01:14:21.000


In [23]:
# sample to make this computationally approachable 
print(len(reviews_small.user_id.unique())) 

# TODO: add this to the pipeline as a variable
sample_n = 1000
sampled_users = users_small.sample(sample_n)
reviews_sampled = reviews_small[reviews_small.user_id.isin(sampled_users.user_id)]
print(len(reviews_sampled.user_id.unique())) 
print(len(reviews_sampled))

193793
1000
21241


In [28]:
reviews_sampled.to_parquet("../data/2023/reviews_small.parquet")

In [24]:
items_small = items[items.parent_asin.isin(reviews_sampled.parent_asin.unique())]
items_small

,title,average_rating,rating_number,description,price,images,parent_asin
102,KHOMO - iPad 2 3 and 4 Generation Case - DUAL ...,4.5,2745,[KHOMO dual Series case choom introduces its b...,11.95,[{'hi_res': 'https://m.media-amazon.com/images...,B06XKRXLDR
140,HGST Touro Desk Pro 3TB USB 3.0 External Hard ...,3.2,15,"[Product Description, Hitachi Touro Desk Pro 3...",None,[{'hi_res': 'https://m.media-amazon.com/images...,B004W7DR02
153,Tiffen 77mm Photo Essentials Kit with UV Prote...,4.6,576,"[Product Description, This kit consists of a: ...",89.99,"[{'hi_res': None, 'large': 'https://m.media-am...",B0067HY1EQ
190,"Technical Pro C-QS-1225 25' 1/4"" to Speakon Sp...",4.4,11,[Available in 12 and 16 gauge speaker wire. Av...,15.95,[{'hi_res': 'https://m.media-amazon.com/images...,B0029L7IYY
234,CaseBot Leather Case for Kindle Oasis (10th an...,4.5,1444,"[CaseBot, a premium series brought to you by F...",7.99,[{'hi_res': 'https://m.media-amazon.com/images...,B07M67FJDB
...,...,...,...,...,...,...,...
1380025,MONICONA Air Tube Earpiece Surveillance Kit Ea...,4.0,35,[],29.99,[{'hi_res': 'https://m.media-amazon.com/images...,B09N8YFX24
1380045,"Galaxy Tab S2 8.0 Case, IVSO Samsung Galaxy Ta...",3.7,62,[],None,"[{'hi_res': None, 'large': 'https://m.media-am...",B00UP5Q67Q
1380090,Drop Protected Case Cover Compatible with Airp...,4.1,426,[],None,[{'hi_res': 'https://m.media-amazon.com/images...,B08CBV77FD
1380577,Logitech iPad Pro 10.5 inch Keyboard Case | SL...,4.3,1581,[Slim Combo is the ultimate companion to your ...,None,[{'hi_res': 'https://m.media-amazon.com/images...,B0753223KN


In [27]:
items_small.to_parquet("../data/2023/items_small.parquet")